In [ ]:
import pandas as pd

# 指定Excel文件路径
excel_file_path = '00_raw_data\\1_test_zb_df_2t.xlsx'

# 读取Excel文件,并自动设置行索引和列索引-
df = pd.read_excel(excel_file_path, index_col=0)

# 查看数据
print(df)

In [ ]:
#提取偶数列，属性的权重列，并将偶数列（权重列）删除，返回删除权重后的dataframe
def getWeight(df):
    #获取偶数列列名
    even_columns = df.columns[1::2]
    #提取偶数列的数据
    even_columns_data = df[even_columns]    
    #删除偶数列的数据
    newdf = df.drop(columns=even_columns)
    
    #将偶数列的列索引改成和奇数列索引一样
    even_columns_data.set_axis(newdf.columns, axis=1, inplace=True)

    # python可以有多个返回值，如果，只对一个感兴趣，
    # 可以用_来接受，表示不感兴趣
    return even_columns_data, newdf

#data_weight, data = getWeight(df)
#print(data)
#print(data_weight)


In [ ]:
#对数据进行正则化处理
def data_Normalization(df, weight_data):
    #选出值全为0和值全为100的列
    columns_to_drop = df.columns[(df == 0).all() | (df == 100).all()]
    # 创建新的DataFrame，包含有效的列
    new_df = df.drop(columns=columns_to_drop)
    # 删除权重对应的列
    weight = weight_data.drop(columns = columns_to_drop)

    # 对整个dataframe进行归一化，归一化的方式是所有数据除以100
    normalized_df = new_df / 100
    return normalized_df, weight


#normalization_df, weight = data_Normalization(data, data_weight)

In [160]:

file_path = '01_normalization_data\\1_test_zb_df_2t.xlsx'
def saveExcel(weight, normalization_df, file_path):
    #对权重进行去重
    weight = weight.drop_duplicates()
    result_data = pd.concat([weight, normalization_df], ignore_index=True)

    #将权重加入到数据的第一行
    result = result_data.rename(index={0: 'weight'})
    print(result)
    #保存
    result.to_excel(file_path, index=True)
    return result

#saveExcel(weight, normalization_df, file_path)

In [161]:
#定义列表，对三个文件进行批量处理
raw_file = [
     '00_raw_data/1_test_zb_df_2 t.xlsx',
     '00_raw_data/2_test_zb_df_2 t.xlsx',
     '00_raw_data/3_test_zb_df_2 t.xlsx',
     '00_raw_data/人员数据.xlsx'
]

result_file = [
    '01_normalization_data/01_test_zb_df_2 t.xlsx',
    '01_normalization_data/02_test_zb_df_2 t.xlsx',
    '01_normalization_data/03_test_zb_df_2 t.xlsx',
    '01_normalization_data/人员数据.xlsx'
]

sum_data = []
#定义数据处理的函数
def dataMung(raw_path, result_path):
    # 读取Excel文件,并自动设置行索引和列索引-
    df = pd.read_excel(raw_path, index_col=0)
    data_weight, data = getWeight(df)
    normalization_df, weight = data_Normalization(data, data_weight)
    result = saveExcel(weight, normalization_df, result_path)
    sum_data.append(result)

for i in range(3):
    dataMung(raw_file[i], result_file[i])
#dataMung('00_raw_data/人员数据.xlsx', '01_normalization_data/people.xlsx')
    

        XNZJF01000001  XNZJF01000006  XNZJF01060063  XNZJF02010004  \
weight          1.361            2.4            0.6          0.215   
1               0.100            0.5            0.6          0.800   
2               0.100            0.5            0.6          0.800   
3               0.100            0.5            0.6          0.800   
4               0.100            0.5            0.6          0.800   
...               ...            ...            ...            ...   
9997            0.100            0.5            0.6          0.800   
9998            0.100            0.5            0.6          0.800   
9999            0.100            0.5            0.6          0.800   
10000           0.100            0.5            0.6          0.800   
10001           0.100            0.5            0.6          0.800   

        XNZJF02010006  XNZJF02010009  XNZJF02020001  XNZJF02020002  \
weight          2.144          1.074            1.6          1.588   
1               0.5

In [164]:
#根据方差对数据进行聚类
import numpy as np
from sklearn.cluster import KMeans

data_1 = sum_data[0]
#print(data_1)
# 1. 去掉DataFrame中的第一行数据
data_1 = data_1.iloc[1:]

# 2. 对DataFrame的列数据集按照每一列的标准差进行排序
sorted_columns = data_1.std().sort_values(ascending=False).index

# 3. 选择标准差最大的20列数据作为聚类的中心
selected_columns = sorted_columns[:20]
selected_data = data_1[selected_columns]

# 转换为NumPy数组
initial_centers = selected_data.to_numpy()
X = data_1.to_numpy()

# 4. 创建KMeans对象，设置聚类中心的个数
n_clusters = 20  # 假设聚类中心的个数为5
kmeans = KMeans(n_clusters=n_clusters, init=initial_centers)

# 执行聚类
kmeans.fit(X)

# 获取聚类中心
centroids = kmeans.cluster_centers_

# 获取每个样本所属的簇
labels = kmeans.labels_

print("聚类中心：")
print(centroids)
print("样本的簇标签：")
print(labels)

c:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1307: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)


ValueError: The shape of the initial centers (10001, 20) does not match the number of clusters 20.